In [1]:
%pip install langchain
%pip install langchain-openai
%pip install langchain-community
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

api_key_1 = os.environ.get("OPENAI_API_KEY")
api_key_2 = os.environ.get("SERP_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key_1
os.environ["SERP_API_KEY"] = api_key_2

In [4]:
from langchain_openai import OpenAIEmbeddings

openai_embed_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import gzip
import json

wikipedia_filepath = '/workspaces/QA-RAG-LangChain/simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding = 'utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        docs .append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3])
        })

print("Passages:", len(docs))

Passages: 169597


In [6]:
docs = [doc for doc in docs for x in ['linguistics', 'india', 'cheetah']
                if x in doc ['data'].lower().split()]

In [7]:
len(docs)

1364

In [8]:
docs[:3]

[{'metadata': {'title': 'Kurgan hypothesis', 'article_id': '72554'},
  'data': 'The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'},
 {'metadata': {'title': 'Marija Gimbutas', 'article_id': '72558'},
  'data': 'Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'},
 {'metadata': {'title': 'Basil', 'article_id': '73985'},
  'data': 'Basil ("Ocimum basilic

In [ ]:
%pip install -qU langchain langchain-core langchain-community
from langchain_core.documents import Document

docs = [Document(page_content=doc['data'],
                metadata=doc['metadata']) for doc in docs]

Note: you may need to restart the kernel to use updated packages.


In [10]:
docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [11]:
len(docs)

1364

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [13]:
chunked_docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [14]:
len(chunked_docs)

1388

In [15]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name = 'rag_wikipedia_db',
                                  embedding = openai_embed_model,
                                  collection_metadata = {"hnsw:space": "cosine"},
                                  persist_directory= "./wikipedia_db")

In [16]:
chroma_db = Chroma(persist_directory="./wikipedia_db",
                   collection_name='rag_wikipedia_db',
                   embedding_function=openai_embed_model)

In [17]:
chroma_db

In [18]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI (model_name= 'gpt=3.5-turbo', temperature = 0)